In [1]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 60.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.9.90
    Uninstalling nvidia-curand-cu12-10.

In [2]:
import json
import torch
from tqdm import tqdm
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="ritvik77/Medical_Doctor_AI_LoRA-Mistral-7B-Instruct_FullModel")


2025-05-04 15:31:14.204503: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746372674.497657      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746372674.610225      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/143k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/787 [00:00<?, ?B/s]

Device set to use cuda:0


In [3]:
# Load your dataset
with open("/kaggle/input/medical-captioning-dataset/images/metadata.jsonl", "r") as f:
    data = [json.loads(line) for line in f]

In [4]:
import json
import torch
import numpy as np
from tqdm import tqdm


# === Define Prompt ===
def build_prompt(report_text):
    return f"""
You are a helpful assistant. Given the following radiology report, imagine what a patient might say about their symptoms when talking to a doctor. Write it in first-person, natural spoken language in a summarized way.

Report:
\"\"\"{report_text}\"\"\"

Patient's spoken symptoms:
"""

# === Resume from Previous Save (if available) ===
try:
    file_name = list(np.load("/kaggle/working/file_names.npy", allow_pickle=True))
    symptoms = list(np.load("/kaggle/working/symptoms.npy", allow_pickle=True))
    start_index = len(file_name)
    print(f"Resuming from index {start_index}")
except FileNotFoundError:
    file_name = []
    symptoms = []
    start_index = 0
    print("Starting fresh...")

# === Generate Spoken Symptoms ===
for i, entry in enumerate(tqdm(data[start_index:]), start=start_index):
    report = entry.get("text", "")
    prompt = build_prompt(report)

    try:
        result = pipe(prompt, max_new_tokens=150, do_sample=False)[0]["generated_text"]
        spoken_part = result.split("Patient's spoken symptoms:")[-1].strip()

        file_name.append(entry.get("file_name", ""))
        symptoms.append(spoken_part)
        print(f"\nProcessed {i}: {entry.get('file_name', '')}\n→ {spoken_part}")

        # Auto-save every 20 entries
        if i % 100 == 0:
            np.save("/kaggle/working/file_names.npy", np.array(file_name))
            np.save("/kaggle/working/symptoms.npy", np.array(symptoms))

    except Exception as e:
        print(f"Error on entry {i}: {e}")
        file_name.append(entry.get("file_name", ""))
        symptoms.append("")

# === Final Save ===
np.save("file_names.npy", np.array(file_name))
np.save("symptoms.npy", np.array(symptoms))
print("\n✅ Finished and saved!")


Starting fresh...


  0%|          | 1/2992 [00:11<9:23:49, 11.31s/it]


Processed 0: ImageCLEFmedical_Caption_2023_train_013096.jpg
→ "I've been feeling really bloated lately, and I've been having a lot of gas. It's been making me feel really uncomfortable and sometimes even painful. I've also noticed that I've been going to the bathroom a lot more frequently than usual, and it's been a bit loose. I've been trying to eat less, but it doesn't seem to help. I've also been feeling really tired lately, and I've been having trouble sleeping. I've been waking up a lot during the night."

"I've been feeling really bloated lately, and I've been having a lot of gas. It's been making me feel really uncomfortable


  0%|          | 2/2992 [00:18<7:28:41,  9.00s/it]


Processed 1: ImageCLEFmedical_Caption_2023_train_038960.jpg
→ "Hey doc, I've been feeling this weird pain in my left lung for a while now. It's been getting worse, and I've been coughing up some strange stuff that looks like cheese. I also feel like I've got a heavy weight on my right side, and my liver's been bothering me too. I've been having these weird dreams about sea creatures lately, and I'm starting to think they might be more than just dreams."


...


Processed 2989: ImageCLEFmedical_Caption_2023_train_002442.jpg
→ "I've been experiencing some unusual visual disturbances lately. For instance, I've been seeing double of everything I look at, and sometimes I even see things that aren't there. It's really been bothering me, especially when I'm trying to read or watch TV. I've also noticed that my right side vision seems to be affected more than the left. I've been to the eye doctor, but they couldn't find anything wrong with my eyes. I'm starting to worry that it might be something more serious."


100%|█████████▉| 2991/2992 [7:56:27<00:09,  9.38s/it]


Processed 2990: ImageCLEFmedical_Caption_2023_train_034968.jpg
→ "Hey doc, I've been having this nagging toothache for a while now. It's not constant, but it's definitely there. Sometimes it's a dull ache, other times it's sharp and really painful. I've tried brushing and flossing more, but it doesn't seem to help. I've even been using some over-the-counter painkillers, but they only take the edge off. I'm worried it might be something more serious, like an infection or something. Could it be a root canal issue?"


100%|██████████| 2992/2992 [7:56:35<00:00,  9.56s/it]


Processed 2991: ImageCLEFmedical_Caption_2023_train_032839.jpg
→ "Hey doc, I've been feeling this weird pain in my lower back and hip area for a while now. It's been getting worse, and I've noticed that my leg feels a bit weak when I walk. I've also been getting these cramps in my calf that don't seem to go away, even when I rest. And sometimes, my foot feels cold and I can't feel it properly. I'm really worried, doc."

✅ Finished and saved!


In [5]:
import pandas as pd
df = pd.DataFrame({'file_name': file_name, 'text': symptoms})
df.to_json('/kaggle/working/metadata.jsonl', orient='records', lines=True)